In [2]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

#from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import  kernels_reparametrization_space as kernels_repar_space
from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data2
from GEMS_TCO import alg_optimization,  BaseLogger
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [4]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map, offset = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[-3, -1],      
lon_range=[121.0, 131.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

--- Global Monthly Mean for 2024-7: 256.4701 ---


Load daily data applying max-min ordering

In [5]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []      

for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float, # or just omit it 
    keep_ori= False
    )

    daily_aggregated_tensors.append( day_aggregated_tensor )
    daily_hourly_maps.append( day_hourly_map )

print(daily_aggregated_tensors[0].shape)
print(daily_aggregated_tensors[0][:10])

ValueError: operands could not be broadcast together with shapes (7155,) (2,) 

# fit l-bfgs

In [17]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

In [18]:
import torch
import numpy as np
import torch.optim as optim
import time
# NOTE: Assuming the necessary classes and global variables are defined elsewhere, 
# specifically the referenced 'fit_vecchia_lbfgs' class and 'kernels_reparam_space_time' module.

# --- Assume global variables are set: ---
# daily_hourly_maps, daily_aggregated_tensors, nns_map
# lat_lon_resolution, v, mm_cond_number, nheads
# lr, patience, factor, epochs

# --- L-BFGS SPECIFIC GLOBAL PARAMETERS ---
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 7       # Number of outer optimization steps
LBFGS_HISTORY_SIZE = 100   # Memory for Hessian approximation
LBFGS_MAX_EVAL = 50        # Max evaluations (line search) per step

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] # 0 index 
for day_idx in day_indices:  

    # Assuming data access is correct
    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Parameter Initialization (SPATIO-TEMPORAL) ---
    '''  
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_range_time = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    '''
    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.247
    init_range_time = 0.7
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689

    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 7-parameter spatio-temporal list (Log/Linear)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log(phi4)
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (linear)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (linear)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log(nugget)
    ]

    # --- Define parameter groups ---
    lr_all = LBFGS_LR
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    # L-BFGS requires the parameters to be iterable in a single list or group
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info (using placeholder print variables) ---

    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { daily_aggregated_tensor.shape[0]/8}, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the L-BFGS Class ---
    # NOTE: Assuming fit_vecchia_lbfgs is available via kernels_reparam_space_time
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 Set L-BFGS Optimizer ---
    # L-BFGS specific arguments are passed here
    optimizer_vecc = model_instance.set_optimizer(
                params_list,     
                lr=LBFGS_LR,            
                max_iter=LBFGS_MAX_EVAL,        
                history_size=LBFGS_HISTORY_SIZE 
            )

    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer_vecc,
            # covariance_function argument is GONE
            max_steps=LBFGS_MAX_STEPS, 
            grad_tol=1e-7
        )
    

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} L-BFGS steps.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")



--- Starting Day 1 (2024-07-1) ---
Data size per day: 18126.0, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -2.5562
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 2400, Batched Tails: 142608
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/7 / Loss: 1.243622 ---
  Param 0: Value=4.2867, Grad=-5.3208817852082833e-08
  Param 1: Value=1.7396, Grad=-5.134866304769628e-08
  Param 2: Value=0.4891, Grad=-8.514844787937351e-07
  Param 3: Value=-3.7771, Grad=-1.9466146557579975e-08
  Param 4: Value=0.0205, Grad=-3.2878709927942028e-06
  Param 5: Value=-0.1641, Grad=3.015833070129032e-08
  Param 6: Value=-12.1476, Grad=7.248147498080532e-10
  Max Abs Grad: 3.287871e-06
------------------------------
--- Step 2/7 / Loss: 1.241928 ---
  Param 0: Value=4.2867, Grad=-5.3208817852082833e-08
  Param 1: Value=1.7396, Grad=-5.134866304769628e-08

In [ ]:
# --- L-BFGS SPECIFIC GLOBAL PARAMETERS ---
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 10       # Number of outer optimization steps
LBFGS_HISTORY_SIZE = 100   # Memory for Hessian approximation
LBFGS_MAX_EVAL = 50        # Max evaluations (line search) per step

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [1] # 0 index 
for day_idx in day_indices:  

    # Assuming data access is correct
    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Parameter Initialization (SPATIO-TEMPORAL) ---
    '''  
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_range_time = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    '''
    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.247
    init_range_time = 0.7
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689

    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 7-parameter spatio-temporal list (Log/Linear)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log(phi4)
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (linear)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (linear)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log(nugget)
    ]

    # --- Define parameter groups ---
    lr_all = LBFGS_LR
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    # L-BFGS requires the parameters to be iterable in a single list or group
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info (using placeholder print variables) ---
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { daily_aggregated_tensor.shape[0]/8 }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the L-BFGS Class ---
    # NOTE: Assuming fit_vecchia_lbfgs is available via kernels_reparam_space_time
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 Set L-BFGS Optimizer ---
    # L-BFGS specific arguments are passed here
    optimizer = model_instance.set_optimizer(
            param_groups,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        # max_iter in LBFGS is the line search limit
            history_size=LBFGS_HISTORY_SIZE 
        )


    # --- 💥 Call the L-BFGS Fit Method ---
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_aniso_STABLE_log_reparam, 
            max_steps=LBFGS_MAX_STEPS # Outer loop steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} L-BFGS steps.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")


--- Starting Day 2 (2024-07-2) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 10,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -2.5562
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
--- Starting L-BFGS Optimization ---
--- Step 1/10 / Loss: 1.700210 ---
  Param 0: Value=3.7896, Grad=-1.1735791810950357e-06
  Param 1: Value=1.3146, Grad=-5.177567866476432e-07
  Param 2: Value=0.6290, Grad=7.6302044784969e-07
  Param 3: Value=-2.7640, Grad=-5.4658081165455354e-08
  Param 4: Value=0.0023, Grad=-8.187702470612921e-07
  Param 5: Value=-0.1696, Grad=3.6728984261623678e-06
  Param 6: Value=-1.4066, Grad=-1.0927744412572334e-05
  Max Abs Grad: 1.092774e-05
------------------------------
--- Step 2/10 / Loss: 1.686152 ---
  Param 0: Value=3.7896, Grad=-1.1735791810950357e-06
  Param 1: Value=1.3146, Grad=-5.177567866476432e-07
  Param 2: Value=0.6290, Grad=7.6302044784969e-07
  Param 3: Value=-2.7640, Grad=-5.4658081165455354e-08
  

KeyboardInterrupt: 

--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -2.5562
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
--- Starting L-BFGS Optimization ---
--- Step 1/10 / Loss: 1.099366 ---
  Param 0: Value=9.1906, Grad=0.0
  Param 1: Value=21.8702, Grad=0.0
  Param 2: Value=-9.1816, Grad=0.0
  Param 3: Value=-5.1759, Grad=0.0
  Param 4: Value=0.8510, Grad=0.0
  Param 5: Value=3.9055, Grad=0.0
  Param 6: Value=-56.1134, Grad=-1.5296128129348282e-08
  Max Abs Grad: 1.529613e-08
------------------------------

Converged on gradient norm (max|grad| < 1e-05) at step 1
FINAL STATE: Step 1, Loss: 1.0993658256399734
  Raw (vecc) Parameters: [9.190602269069139, 21.87017839865902, -9.181575702042394, -5.1759395277982945, 0.8510437730610125, 3.9054716223330805, -56.11340102891027]
  Interpretable Parameters:
    sigma_sq  : 0.000003
    range_lon : 0.000000
    range_lat : 0.000000
    range_time: 0.000000
    advec_lat : 0.851044
    advec_lon : 3.905472
    nugget    : 0.000000
Day 1 optimization finished in 2311.03s over 1 L-BFGS steps.
Day 1 final results (raw params + loss): [9.190602269069139, 21.87017839865902, -9.181575702042394, -5.1759395277982945, 0.8510437730610125, 3.9054716223330805, -56.11340102891027, 1.0993658256399734]


for day2, it won't converge even after 180 minutes

Hyper parameters for adams optimization

In [ ]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

In [ ]:
# --- End Placeholders ---

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                
    init_phi1 = init_sigmasq * init_phi2            
    init_phi3 = (init_range_lon / init_range_lat)**2  
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 REVISED: Set lr_fast from the global lr
    lr_fast = lr 
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    # Assuming lat_lon_resolution, v, mm_cond_number are defined globally
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the Optimizer Method with Plateau settings ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        # Pass patience
            factor=factor             # Pass factor
        )

    # --- Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_vecc_scheduler_fullbatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, 
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")

In [ ]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
        model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
        
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_SPATIAL_log_reparam)
        vecchia_nll = model_instance.vecchia_space_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll



a = [3.5764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.6764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


a = [3.4764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


Full Negative Log-Likelihood: (22936.561132578532, 22882.89402724899)
Full Negative Log-Likelihood: (22940.44941805631, 22893.58097275657)
Full Negative Log-Likelihood: (22959.63309778168, 22897.341313474775)
